S24 (). La 3-upla de palabras (no letras) más común en los comentarios de los tips. De ser necesario, se puede utilizar un sample y no la totalidad de datos.

# Carga de archivos

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt install openjdk-8-jdk-headless -qq

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
import re

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"10J1v6eKps-2xslC-X4voNOZ0a3zLNoF0"})   # replace the id with id of file you want to access
downloaded.GetContentFile('tip.csv') 

In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('tip.csv', header=True).select(['text']).dropna()
rdd = df.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


# Calculo

Defino una funcion para calcular 3-uplas de un string

In [ ]:
def tresUplas(s):
  l = s.split()
  r = []
  for i in range(len(l) - 2):
    r.append("{} {} {}".format(l[i], l[i+1], l[i+2]))
  return r

1- Le saco los simbolos especiales a las reviews

2- Reemplazo la coma por un espacio

3- Aplico un flatMap agrupando de a 3 

4- Le agrego un 1 a cada registro para poder sumar

5- Hago un reduceByKey para poder sumar las ocurrencias de cada upla 

6- Hago un reduce y obtengo el registro mas alto


In [ ]:
rdd = rdd.map(lambda x: re.sub("[$?@&!.']", "", x[0].lower()))\
         .map(lambda x: x.replace(",", " "))\
         .flatMap(lambda x: tresUplas(x))\
         .map(lambda x: (x,1))\
         .reduceByKey(lambda x,y: x+y)

In [ ]:
moreFrequently = rdd.reduce(lambda x,y : x if x[1]>y[1] else y)

In [ ]:
moreFrequently


('love this place', 9765)